# Class Notes: Graph Data Structures in Python

## What is a Graph?
- A graph is a set of **nodes** (vertices) connected by **edges**.
- Nodes can be labeled (e.g., 1, 2, 3 or A, B, C).
- Edges might be **undirected** (two-way connection) or **directed** (one-way connection).
- **Weighted graphs** have edges with associated weights (e.g., cost, distance).
- **Special graph types**
  - **Tree**: A type of graph with no cycles.

## Graph Representations
- **Adjacency List**: Each node stores its neighbors in a list or set.
- **Adjacency Matrix**: A 2D matrix with 1 (edge exists) or 0 (edge absent) for all node pairs; can store weights.

## Use Cases
- Network analysis, path finding, graph neural networks, etc.

## Goals of the Implementation
- Create a `Graph` class with key functionalities:
  - Add/remove nodes and edges
  - Support for directed/undirected and weighted graphs
  - Get neighbors, check for node/edge existence
  - Breadth-First Search (BFS)
  - Depth-First Search (DFS)
  - (Paste-in) Dijkstra’s algorithm, shortest path, adjacency matrix converter

## Key Methods in the `Graph` class
- `__init__` — initialize graph, option for directed
- `__repr__` — print friendly adjacency list
- `add_node`, `remove_node`, `add_edge`, `remove_edge`
- `get_neighbors`, `has_node`, `has_edge`, `get_nodes`, `get_edges`
- `bfs`, `dfs`
- Additional (pasted): Dijkstra’s, shortest path, adjacency matrix conversion.


## Example Graph
Weighted, undirected graph with nodes A, B, C, D, E, F, G, H, I. Example edges:
- A - B (1), A - C (10), B - C (1), ...

## Summary
- BFS uses a queue (FIFO), visits neighbors layer by layer.
- DFS uses a stack (LIFO), dives deep along paths.
- Dijkstra’s algorithm finds shortest paths (code shared but not explained in video).

In [ ]:
class Graph:
    def __init__(self, directed=False):
        self.directed = directed
        self.adjacency_list = {}

    def __repr__(self):
        graph_str = ""
        for node, neighbors in self.adjacency_list.items():
            graph_str += f"{node} -> {neighbors}\n"
        return graph_str

    def add_node(self, node):
        if node not in self.adjacency_list:
            self.adjacency_list[node] = set()
        else:
            raise ValueError("Node already exists.")

    def remove_node(self, node):
        if node not in self.adjacency_list:
            raise ValueError("Node does not exist.")
        for neighbors in self.adjacency_list.values():
            neighbors.discard(node)
        del self.adjacency_list[node]

    def add_edge(self, from_node, to_node, weight=None):
        if from_node not in self.adjacency_list:
            self.add_node(from_node)
        if to_node not in self.adjacency_list:
            self.add_node(to_node)
        if weight is None:
            self.adjacency_list[from_node].add(to_node)
            if not self.directed:
                self.adjacency_list[to_node].add(from_node)
        else:
            self.adjacency_list[from_node].add((to_node, weight))
            if not self.directed:
                self.adjacency_list[to_node].add((from_node, weight))

    def remove_edge(self, from_node, to_node):
        if from_node in self.adjacency_list and to_node in self.adjacency_list[from_node]:
            self.adjacency_list[from_node].remove(to_node)
            if not self.directed and to_node in self.adjacency_list:
                if from_node in self.adjacency_list[to_node]:
                    self.adjacency_list[to_node].remove(from_node)
        else:
            raise ValueError("Edge does not exist.")

    def get_neighbors(self, node):
        return self.adjacency_list.get(node, set())

    def has_node(self, node):
        return node in self.adjacency_list

    def has_edge(self, from_node, to_node):
        if from_node in self.adjacency_list:
            return to_node in self.adjacency_list[from_node]
        return False

    def get_nodes(self):
        return list(self.adjacency_list.keys())

    def get_edges(self):
        edges = []
        for from_node, neighbors in self.adjacency_list.items():
            for to_node in neighbors:
                edges.append((from_node, to_node))
        return edges

    def bfs(self, start):
        visited = set()
        queue = [start]
        order = []
        while queue:
            node = queue.pop(0)
            if node not in visited:
                visited.add(node)
                order.append(node)
                neighbors = self.get_neighbors(node)
                for neighbor in neighbors:
                    # If edges are weighted, neighbor might be a tuple
                    if isinstance(neighbor, tuple):
                        neighbor = neighbor[0]
                    if neighbor not in visited:
                        queue.append(neighbor)
        return order

    def dfs(self, start):
        visited = set()
        stack = [start]
        order = []
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                order.append(node)
                neighbors = self.get_neighbors(node)
                for neighbor in sorted(neighbors, reverse=True):
                    if isinstance(neighbor, tuple):
                        neighbor = neighbor[0]
                    if neighbor not in visited:
                        stack.append(neighbor)
        return order

In [ ]:
g = Graph()
g.add_edge('A', 'B', 1)
g.add_edge('A', 'C', 10)
g.add_edge('B', 'C', 1)
g.add_edge('B', 'D', 1)
g.add_edge('D', 'C', 1)
g.add_edge('A', 'E', 1)
g.add_edge('E', 'F', 1)
g.add_edge('G', 'F', 1)
g.add_edge('F', 'H', 1)
g.add_edge('H', 'I', 1)
g.add_edge('I', 'G', 100)

print(g)
print("BFS from A:", g.bfs('A'))
print("DFS from A:", g.dfs('A'))

A -> {('C', 10), ('E', 1), ('B', 1)}
B -> {('A', 1), ('C', 1), ('D', 1)}
C -> {('A', 10), ('B', 1), ('D', 1)}
D -> {('C', 1), ('B', 1)}
E -> {('A', 1), ('F', 1)}
F -> {('E', 1), ('H', 1), ('G', 1)}
G -> {('F', 1), ('I', 100)}
H -> {('I', 1), ('F', 1)}
I -> {('H', 1), ('G', 100)}

BFS from A: ['A', 'C', 'E', 'B', 'D', 'F', 'H', 'G', 'I']
DFS from A: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'H']


# Step-by-Step Graph Construction and Traversal

## 1. Building the Graph

Let's add each edge and visualize the graph as it grows:

### Step 1: Add edge A-B (weight 1)
```
A --1-- B
```

### Step 2: Add edge A-C (weight 10)
```
A --1-- B
|
10
|
C
```

### Step 3: Add edge B-C (weight 1)
```
A --1-- B
|     /
10  1
| /
C
```

### Step 4: Add edge B-D (weight 1)
```
A --1-- B --1-- D
|     /
10  1
| /
C
```

### Step 5: Add edge D-C (weight 1)
```
A --1-- B --1-- D
|     /      /
10  1     1
| /      /
C -------
```

### Step 6: Add edge A-E (weight 1)
```
A --1-- B --1-- D
|     /      /
10  1     1
| /      /
C -------
|
1
E
```

### Step 7: Add edge E-F (weight 1)
```
A --1-- B --1-- D
|     /      /
10  1     1
| /      /
C -------
|
1
E --1-- F
```

### Step 8: Add edge G-F (weight 1)
```
A --1-- B --1-- D
|     /      /
10  1     1
| /      /
C -------
|
1
E --1-- F --1-- G
```

### Step 9: Add edge F-H (weight 1)
```
A --1-- B --1-- D
|     /      /
10  1     1
| /      /
C -------
|
1
E --1-- F --1-- G
        |
        1
        H
```

### Step 10: Add edge H-I (weight 1)
```
...
        F --1-- G
        |
        1
        H --1-- I
```

### Step 11: Add edge I-G (weight 100)
```
...
        F --1-- G
        |      ^
        1      |
        H --1-- I
             100|
```

## 2. Final Graph Structure
```
   1      1
A----B----D
|  / |   /
10 1 | 1
|/   | /
C----/ 
|
1
E
|
1
F --1-- G
|
1
H --1-- I
 \____100_/
```

## 3. Traversals

### BFS from A
- **Order:** A, B, C, E, D, F, G, H, I
- **Explanation:**
  1. Start at A (enqueue neighbors B, C, E)
  2. Visit B (enqueue D)
  3. Visit C (no new neighbors)
  4. Visit E (enqueue F)
  5. Visit D (no new neighbors)
  6. Visit F (enqueue G, H)
  7. Visit G (no new neighbors)
  8. Visit H (enqueue I)
  9. Visit I (no new neighbors)

### DFS from A
- **Order:** A, B, C, D, E, F, G, H, I (may vary depending on neighbor order)
- **Explanation:**
  1. Start at A, go to B
  2. From B, go to C
  3. From C, go to D
  4. Backtrack, visit E
  5. From E, go to F
  6. From F, go to G
  7. Backtrack, visit H
  8. From H, go to I

## 4. Adjacency List (as printed)
```
A -> {('B', 1), ('E', 1), ('C', 10)}
B -> {('A', 1), ('C', 1), ('D', 1)}
C -> {('A', 10), ('B', 1), ('D', 1)}
D -> {('B', 1), ('C', 1)}
E -> {('A', 1), ('F', 1)}
F -> {('E', 1), ('G', 1), ('H', 1)}
G -> {('F', 1), ('I', 100)}
H -> {('F', 1), ('I', 1)}
I -> {('H', 1), ('G', 100)}
```